# Machine Learning para DOA - Dregon Dataset

O objetivo aqui é utilizar regrassão para determinar a direção de chegada. Para isso, já extraí os delays referentes a todas as combinações de microfones do Dregon Dataset - Clean Speech. O arquivo se chama dregonDelaysDataset.csv. O algoritmo usado para extrair os delays foi o gerarDatasetReal.py.

Para treinar os regressores, vou utilizar dados fictícios. Uma vez que sei a geometria dos microfones, posso calcular qual deveria ser o delay caso o som viesse de uma determinada direção. O algoritmo usado para gerar os datasets de dados fictícios foi o gerarDatasetFicticio.py. Esses arquivos seguem o seguinte padrão de nomeação:

dregon_ficticio_azimutal_limiteinferior_limitesuperior_elevacao_limiteinferior_limitesuperior_passo_passo.csv

In [1]:
import pandas as pd
import numpy as np
from math import sqrt, cos, sin, pi
from sklearn.linear_model import LinearRegression, LassoLars, Lasso, Ridge, RANSACRegressor, TheilSenRegressor, BayesianRidge
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error, r2_score

## Importação dos dados reais

O dataset tem as seguintes colunas:

Nome do arquivo WAV, Features, Azimutal Real, Elevação Real

In [2]:
datasetReal   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregonDelaysDataset.csv")
xReal         = np.array(datasetReal[datasetReal.columns[1:-2]])
yRealAzimutal = np.array(datasetReal[datasetReal.columns[-2]])
yRealElevacao = np.array(datasetReal[datasetReal.columns[-1]])

In [3]:
print(xReal[0], "\n", yRealAzimutal[0], "\n", yRealElevacao[0])

[ 12  12  22  18  17   7   6  -1  10   6   4  -5  -6  11   7   5  -5  -6
  -5  -6 -16 -17  -1 -11 -12  -9 -10  -1] 
 45 
 -30


## Treinando com dados fictícios - ELEVACAO

Vou criar uma função que já calcula o MSE e o R2 pra deixar o código mais limpo

In [4]:
def aplicarRegressor(objRegressor, xTrain, xTest, yTrain, yTest):
    objRegressor.fit(xTrain, yTrain)
    yPred = objRegressor.predict(xTest)
    
    return mean_squared_error(yTest, yPred), r2_score(yTest, yPred)

### Azimutal: 0 a 90 | Elevacao: -90 a 0 | Passo: 5

In [5]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_90_elevacao_-90_0_passo_5.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioElevacao
yTest  = yRealElevacao

In [6]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi090Ele900Pas5 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi090Ele900Pas5

,Regressor,RMSE,MSE,R2
0,LinearRegression,4.075578e+12,1.661034e+25,-1.107356e+23
1,Ridge,4.914923e+00,2.415646e+01,8.389569e-01
2,Lasso,7.137261e+00,5.094049e+01,6.603967e-01
3,LassoLars,2.082456e+01,4.336624e+02,-1.891083e+00
4,BayesianRidge,4.916172e+00,2.416875e+01,8.388750e-01
5,RANSACRegressor,9.626757e+12,9.267446e+25,-6.178297e+23
6,TheilSenRegressor,4.986340e+00,2.486359e+01,8.342427e-01


### Azimutal: 0 a 90 | Elevacao: -90 a 0 | Passo: 10

In [7]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_90_elevacao_-90_0_passo_10.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioElevacao
yTest  = yRealElevacao

In [8]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi090Ele900Pas10 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi090Ele900Pas10

,Regressor,RMSE,MSE,R2
0,LinearRegression,5.114453e+00,2.615763e+01,8.256158e-01
1,Ridge,5.114690e+00,2.616006e+01,8.255996e-01
2,Lasso,7.514473e+00,5.646730e+01,6.235513e-01
3,LassoLars,8.709523e+00,7.585579e+01,4.942947e-01
4,BayesianRidge,5.118387e+00,2.619788e+01,8.253474e-01
5,RANSACRegressor,4.338533e+12,1.882287e+25,-1.254858e+23
6,TheilSenRegressor,5.174916e+00,2.677976e+01,8.214683e-01


### Azimutal: 0 a 90 | Elevacao: -90 a 0 | Passo: 15

In [9]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_90_elevacao_-90_0_passo_15.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioElevacao
yTest  = yRealElevacao

In [10]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi090Ele900Pas15 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi090Ele900Pas15

,Regressor,RMSE,MSE,R2
0,LinearRegression,5.333727e+00,2.844864e+01,8.103424e-01
1,Ridge,5.334067e+00,2.845227e+01,8.103182e-01
2,Lasso,7.894705e+00,6.232637e+01,5.844909e-01
3,LassoLars,5.629832e+00,3.169501e+01,7.886999e-01
4,BayesianRidge,5.340389e+00,2.851975e+01,8.098683e-01
5,RANSACRegressor,1.247349e+13,1.555879e+26,-1.037253e+24
6,TheilSenRegressor,5.391202e+00,2.906506e+01,8.062330e-01


### Azimutal: 0 a 350 | Elevacao: -90 a 90 | Passo: 5

In [11]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_350_elevacao_-90_90_passo_5.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioElevacao
yTest  = yRealElevacao

In [12]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi0350Ele9090Pas5 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi0350Ele9090Pas5

,Regressor,RMSE,MSE,R2
0,LinearRegression,2.286298e+13,5.227159e+26,-3.484773e+24
1,Ridge,4.565150e+00,2.084059e+01,8.610627e-01
2,Lasso,5.317472e+00,2.827551e+01,8.114966e-01
3,LassoLars,1.936492e+01,3.750000e+02,-1.500000e+00
4,BayesianRidge,4.564736e+00,2.083682e+01,8.610879e-01
5,RANSACRegressor,4.361364e+13,1.902150e+27,-1.268100e+25
6,TheilSenRegressor,4.629935e+00,2.143630e+01,8.570914e-01


### Azimutal: 0 a 350 | Elevacao: -90 a 90 | Passo: 10

In [13]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_350_elevacao_-90_90_passo_10.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioElevacao
yTest  = yRealElevacao

In [14]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi0350Ele9090Pas10 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi0350Ele9090Pas10

,Regressor,RMSE,MSE,R2
0,LinearRegression,5.883465e+13,3.461516e+27,-2.307677e+25
1,Ridge,4.852938e+00,2.355101e+01,8.429933e-01
2,Lasso,5.607939e+00,3.144898e+01,7.903401e-01
3,LassoLars,7.147418e+00,5.108558e+01,6.594295e-01
4,BayesianRidge,4.851121e+00,2.353337e+01,8.431108e-01
5,RANSACRegressor,5.803054e+13,3.367544e+27,-2.245029e+25
6,TheilSenRegressor,4.921266e+00,2.421886e+01,8.385409e-01


### Azimutal: 0 a 350 | Elevacao: -90 a 90 | Passo: 15

In [15]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_350_elevacao_-90_90_passo_15.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioElevacao
yTest  = yRealElevacao

In [16]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi0350Ele9090Pas15 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi0350Ele9090Pas15

,Regressor,RMSE,MSE,R2
0,LinearRegression,1.407053e+14,1.979798e+28,-1.319865e+26
1,Ridge,5.139299e+00,2.641239e+01,8.239174e-01
2,Lasso,5.897144e+00,3.477631e+01,7.681579e-01
3,LassoLars,3.136503e+00,9.837648e+00,9.344157e-01
4,BayesianRidge,5.134883e+00,2.636702e+01,8.242199e-01
5,RANSACRegressor,5.709615e+13,3.259971e+27,-2.173314e+25
6,TheilSenRegressor,5.229379e+00,2.734640e+01,8.176906e-01


## Treinando com dados fictícios - AZIMUTAL

### Azimutal: 0 a 90 | Elevacao: -90 a 0 | Passo: 5

In [17]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_90_elevacao_-90_0_passo_5.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioAzimutal
yTest  = yRealAzimutal

In [18]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi090Ele900Pas5 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi090Ele900Pas5

,Regressor,RMSE,MSE,R2
0,LinearRegression,2.263879e+14,5.125148e+28,-1.822275e+26
1,Ridge,7.234454e+00,5.233732e+01,8.139117e-01
2,Lasso,6.735234e+00,4.536337e+01,8.387080e-01
3,LassoLars,2.095911e+01,4.392842e+02,-5.618993e-01
4,BayesianRidge,7.161898e+00,5.129279e+01,8.176256e-01
5,RANSACRegressor,1.158560e+14,1.342262e+28,-4.772487e+25
6,TheilSenRegressor,7.507530e+00,5.636300e+01,7.995982e-01


### Azimutal: 0 a 90 | Elevacao: -90 a 0 | Passo: 10

In [19]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_90_elevacao_-90_0_passo_10.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioAzimutal
yTest  = yRealAzimutal

In [20]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi090Ele900Pas10 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi090Ele900Pas10

,Regressor,RMSE,MSE,R2
0,LinearRegression,7.039088e+00,4.954876e+01,8.238266e-01
1,Ridge,7.038073e+00,4.953447e+01,8.238774e-01
2,Lasso,6.562015e+00,4.306004e+01,8.468976e-01
3,LassoLars,7.915065e+00,6.264825e+01,7.772507e-01
4,BayesianRidge,6.747765e+00,4.553234e+01,8.381072e-01
5,RANSACRegressor,1.022155e+14,1.044800e+28,-3.714844e+25
6,TheilSenRegressor,7.207677e+00,5.195060e+01,8.152867e-01


### Azimutal: 0 a 90 | Elevacao: -90 a 0 | Passo: 15

In [21]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_90_elevacao_-90_0_passo_15.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioAzimutal
yTest  = yRealAzimutal

In [22]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi090Ele900Pas15 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi090Ele900Pas15

,Regressor,RMSE,MSE,R2
0,LinearRegression,6.827531e+00,4.661518e+01,8.342571e-01
1,Ridge,6.825629e+00,4.658921e+01,8.343495e-01
2,Lasso,6.370380e+00,4.058174e+01,8.557094e-01
3,LassoLars,4.063426e+00,1.651143e+01,9.412927e-01
4,BayesianRidge,6.175217e+00,3.813331e+01,8.644149e-01
5,RANSACRegressor,7.746287e+13,6.000496e+27,-2.133510e+25
6,TheilSenRegressor,6.774557e+00,4.589463e+01,8.368191e-01


### Azimutal: 0 a 350 | Elevacao: -90 a 90 | Passo: 5

In [23]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_350_elevacao_-90_90_passo_5.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioAzimutal
yTest  = yRealAzimutal

In [24]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi0350Ele9090Pas5 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi0350Ele9090Pas5

,Regressor,RMSE,MSE,R2
0,LinearRegression,6.392619e+14,4.086558e+29,-1.452999e+27
1,Ridge,3.849255e+01,1.481676e+03,-4.268183e+00
2,Lasso,4.339874e+01,1.883451e+03,-5.696715e+00
3,LassoLars,7.610737e+01,5.792332e+03,-1.959496e+01
4,BayesianRidge,3.838737e+01,1.473590e+03,-4.239432e+00
5,RANSACRegressor,2.059284e+14,4.240649e+28,-1.507786e+26
6,TheilSenRegressor,4.010236e+01,1.608199e+03,-4.718042e+00


### Azimutal: 0 a 350 | Elevacao: -90 a 90 | Passo: 10

In [25]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_350_elevacao_-90_90_passo_10.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioAzimutal
yTest  = yRealAzimutal

In [26]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi0350Ele9090Pas10 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi0350Ele9090Pas10

,Regressor,RMSE,MSE,R2
0,LinearRegression,8.725603e+14,7.613615e+29,-2.707063e+27
1,Ridge,3.980811e+01,1.584686e+03,-4.634438e+00
2,Lasso,4.502288e+01,2.027060e+03,-6.207325e+00
3,LassoLars,3.358851e+01,1.128188e+03,-3.011335e+00
4,BayesianRidge,3.932175e+01,1.546200e+03,-4.497600e+00
5,RANSACRegressor,2.823626e+14,7.972861e+28,-2.834795e+26
6,TheilSenRegressor,4.180785e+01,1.747896e+03,-5.214742e+00


### Azimutal: 0 a 350 | Elevacao: -90 a 90 | Passo: 15

In [27]:
datasetFicticio   = pd.read_csv("/home/dimi/Programming/IC2019/DOA/Datasets/dregon_ficticio_azimutal_0_350_elevacao_-90_90_passo_15.csv")

# IMPORTANDO O DATASET COMO FLOAT (ORIGINAL)
xFicticio         = np.array(datasetFicticio[datasetFicticio.columns[0:-2]])
yFicticioAzimutal = np.array(datasetFicticio[datasetFicticio.columns[-2]])
yFicticioElevacao = np.array(datasetFicticio[datasetFicticio.columns[-1]])

# SEPARANDO EM DADOS DE TREINAMENTO E TESTE
xTrain = xFicticio
xTest  = xReal
yTrain = yFicticioAzimutal
yTest  = yRealAzimutal

In [28]:
# TREINANDO OS REGRESSORES

regressores = [LinearRegression(), Ridge(), Lasso(), LassoLars(), BayesianRidge(), RANSACRegressor(), TheilSenRegressor()]
data = []

for regressor in regressores:
    mse, r2 = aplicarRegressor(regressor, xTrain, xTest, yTrain, yTest)
    data.append([regressor.__class__.__name__, sqrt(mse), mse, r2])
    
resultadosAzi0350Ele9090Pas15 = pd.DataFrame(data=data, columns=["Regressor", "RMSE", "MSE", "R2"])
resultadosAzi0350Ele9090Pas15

,Regressor,RMSE,MSE,R2
0,LinearRegression,2.090016e+15,4.368168e+30,-1.553126e+28
1,Ridge,4.133992e+01,1.708989e+03,-5.076406e+00
2,Lasso,4.672590e+01,2.183310e+03,-6.762879e+00
3,LassoLars,2.382823e+01,5.677846e+02,-1.018790e+00
4,BayesianRidge,4.011328e+01,1.609076e+03,-4.721158e+00
5,RANSACRegressor,3.485446e+01,1.214833e+03,-3.319407e+00
6,TheilSenRegressor,4.217515e+01,1.778743e+03,-5.324419e+00
